In [2]:
import os
import pandas as pd
import re

# Ruta a la carpeta que contiene los archivos
data_folder = "../data/input/daily_dump"

# Crear una lista vacía para almacenar los DataFrames
df_list = []

# Expresión regular para validar la estructura df_games_YYYYMMDD.csv
pattern = re.compile(r'^df_games_\d{8}\.csv$')

# Recorrer los archivos en la carpeta
for file_name in os.listdir(data_folder):
    # Verificar si el archivo cumple con la estructura df_games_YYYYMMDD.csv usando la expresión regular
    if pattern.match(file_name):
        # Extraer la fecha del archivo
        date_str = file_name[9:17]
        year = int(date_str[:4])
        month = int(date_str[4:6])
        day = int(date_str[6:8])
        
        # Filtrar archivos que están entre el 11 y 30 de septiembre de 2024
        if year == 2024 and month == 9 and 11 <= day <= 30:
            # Leer el archivo y agregarlo a la lista
            file_path = os.path.join(data_folder, file_name)
            df = pd.read_csv(file_path)
            df_list.append(df)

# Concatenar todos los DataFrames en uno solo
if df_list:
    df_consolidated = pd.concat(df_list, ignore_index=True)
    
    # Guardar el DataFrame consolidado como un nuevo archivo CSV
    df_consolidated.to_csv(os.path.join(data_folder, "df_games_sep_test.csv"), index=False)
    print("Archivo consolidado guardado como df_games_sep_test.csv")
else:
    print("No se encontraron archivos en el rango de fechas especificado.")



Archivo consolidado guardado como df_games_sep_test.csv


In [3]:
from src.standarice_data import TenisModelHandler

# Cargar nuevos datos para predicción
df_new = pd.read_csv(os.path.join(data_folder, "df_games_sep_test.csv"), delimiter='|')

# Crear instancia de la clase Preprocessing
preprocessor = TenisModelHandler(df_new, models_path='../models')

# Generar predicciones con el modelo deseado
resultado = preprocessor.predict_values('xgboost')

# Mostrar el DataFrame con las predicciones
print(resultado.head())

                 Location  pl1_bet  pl2_bet        pl1_name       pl1_flag  \
0     Szczecin challenger     1.30     3.31      Andreev A.       Bulgaria   
2     Szczecin challenger     2.81     1.40   Berrettini J.          Italy   
4     Szczecin challenger     4.27     1.20     Vrbensky M.  CzechRepublic   
6  Guangzhou 2 challenger     1.22     4.11  Shimabukuro S.          Japan   
8  Guangzhou 2 challenger     2.41     1.53      Polmans M.      Australia   

   pl1_year_pro  pl1_weight  pl1_height pl1_hand  pl1_age  ...      pl2_flag  \
0        2015.0        66.0       180.0    right     23.0  ...       Germany   
2        2012.0        85.0       191.0    right     25.0  ...         Italy   
4        2016.0        65.0       180.0    right     24.0  ...         Spain   
6        2015.0        75.0       183.0    right     27.0  ...         China   
8        2011.0        78.0       188.0    right     27.0  ...  GreatBritain   

  pl2_year_pro  pl2_weight  pl2_height  pl2_hand p

g:\dev\tennis_bet\code\src\standarice_data.py:397: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2015. 2012. 2012. ... 2016. 2009. 2020.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.df.update(pl1_df_filled)


In [5]:
resultado["ActualWinner"] = "pl1"

In [17]:
resultado["winner_prediction"].value_counts()

winner_prediction
pl1    408
pl2    289
Name: count, dtype: int64

In [20]:
408 / (408+289)

0.5853658536585366

In [12]:
import pandas as pd
import numpy as np
import optuna

# Definir la función objetivo para la optimización
def simulate_betting_strategy(params, resultado, initial_capital=100.0):
    risk_factor, max_daily_bet_fraction, max_bet_fraction_per_match, probability_rule = params
    
    # Asegurar que los parámetros estén dentro de los límites
    if not (0 < risk_factor <= 1):
        return np.inf
    if not (0 < max_daily_bet_fraction <= 1):
        return np.inf
    if not (0 < max_bet_fraction_per_match <= 1):
        return np.inf
    if not (0 < probability_rule <= 1):
        return np.inf
    
    bankroll = initial_capital
    dates = resultado['Date'].unique()
    
    for date in dates:
        day_matches = resultado[resultado['Date'] == date]
        daily_bet_amount = 0
        daily_max_bet = bankroll * max_daily_bet_fraction
        
        for idx, match in day_matches.iterrows():
            if match["winner_prediction"] == "pl1":
                p1_win_prob = match['probability']
            else:
                p1_win_prob = 1 - match['probability']
                
            p1_loss_prob = 1 - p1_win_prob
            odds_p1 = match['pl1_bet']
            b1 = odds_p1 - 1
            f1 = (b1 * p1_win_prob - p1_loss_prob) / b1
            f1 = max(f1, 0) * risk_factor
            f1 = min(f1, 1)
            
            # Cálculo para el Jugador 2
            p2_win_prob = 1 - p1_win_prob
            p2_loss_prob = 1 - p2_win_prob
            odds_p2 = match['pl2_bet']
            b2 = odds_p2 - 1
            f2 = (b2 * p2_win_prob - p2_loss_prob) / b2
            f2 = max(f2, 0) * risk_factor
            f2 = min(f2, 1)
            
            # Decidir por qué jugador apostar
            if f1 > f2 and f1 > 0:
                f = f1
                odds = odds_p1
                player_bet_on = 'pl1'
                win_prob = p1_win_prob
            elif f2 > f1 and f2 > 0:
                f = f2
                odds = odds_p2
                player_bet_on = 'pl2'
                win_prob = p2_win_prob
            else:
                continue  # No apostar si no hay valor positivo de Kelly
            
            if win_prob <= probability_rule:
                continue 
            
            max_bet_amount_per_match = bankroll * max_bet_fraction_per_match
            bet_amount = f * bankroll
            bet_amount = min(bet_amount, max_bet_amount_per_match)
            
            # Asegurar que no se exceda el máximo diario
            if daily_bet_amount + bet_amount > daily_max_bet:
                bet_amount = daily_max_bet - daily_bet_amount
                if bet_amount <= 0:
                    continue
                f = bet_amount / bankroll  # Recalcular fracción de Kelly
            
            daily_bet_amount += bet_amount
            
            # Actualizar capital basado en los resultados
            actual_winner = match['ActualWinner']  # Debes tener esta columna en tu DataFrame
            if actual_winner == player_bet_on:
                payout = bet_amount * odds
                profit = payout - bet_amount
                bankroll += profit
            else:
                bankroll -= bet_amount
            
            if bankroll <= 0:
                return np.inf  # Quiebra, resultado no deseado
    
    # Devolver el capital final negativo para minimizar
    return -bankroll

# Función objetivo para Optuna
def objective(trial):
    risk_factor = trial.suggest_uniform('risk_factor', 0.1, 0.98)
    max_daily_bet_fraction = trial.suggest_uniform('max_daily_bet_fraction', 0.05, 0.90)
    max_bet_fraction_per_match = trial.suggest_uniform('max_bet_fraction_per_match', 0.05, 0.50)
    probability_rule = trial.suggest_uniform('probability_rule', 0.20, 0.97)
    
    params = [risk_factor, max_daily_bet_fraction, max_bet_fraction_per_match, probability_rule]
    final_capital = simulate_betting_strategy(params, resultado)
    
    return final_capital

# Crear estudio y optimizar
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=3000)

# Mostrar los mejores parámetros
print("Mejores parámetros encontrados:")
trial = study.best_trial

print(f"  Risk Factor: {trial.params['risk_factor']:.4f}")
print(f"  Max Daily Bet Fraction: {trial.params['max_daily_bet_fraction']:.4f}")
print(f"  Max Bet Fraction per Match: {trial.params['max_bet_fraction_per_match']:.4f}")
print(f"  Probability Rule: {trial.params['probability_rule']:.4f}")
print(f"  Capital Final: {-trial.value:.2f} USD")


[I 2024-10-02 19:39:25,692] A new study created in memory with name: no-name-10a8db8c-2791-4d76-95f5-cd724d757ab8
C:\Users\Julián Pérez\AppData\Local\Temp\ipykernel_10416\3449184236.py:96: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  risk_factor = trial.suggest_uniform('risk_factor', 0.1, 0.98)
C:\Users\Julián Pérez\AppData\Local\Temp\ipykernel_10416\3449184236.py:97: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  max_daily_bet_fraction = trial.suggest_uniform('max_daily_bet_fraction', 0.05, 0.90)
C:\Users\Julián Pérez\AppData\Local\Temp\ipykernel_10416\3449184236.py:98: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optu

Mejores parámetros encontrados:
  Risk Factor: 0.9799
  Max Daily Bet Fraction: 0.8996
  Max Bet Fraction per Match: 0.4851
  Probability Rule: 0.9522
  Capital Final: 122.13 USD
